Change from 4.0

- deseason before LSTM like notebook 5.1

For reference, check https://github.com/drivendata/benchmarks/blob/master/dengue-benchmark-statsmodels.ipynb


Submissions

- 4.0.1A LSTM with deseasoning on all features, no polynomial features except intercept
- 4.0.1B ditto with feature subset from notebook 3.0
  - ditto from notebook 3.0.1 ... did not predict epidemic end of 2010 .. not submitting
- 4.0.1C lahead=60, batchsize=64, feature subset from notebook 3.0.1
- 4.0.1D ditto with all features
  - without log10, end-of-2010 epidemic lasts a bit longer into 2011
  - with log10(target+1)
  - dropped `*_resid` for all features + kept `weekofyear_original` among `weekofyear_*` features
  - submit score=24
- 4.0.1E ditto with lstm(100) instead of lstm(10)
  - submission score = 25
  

TODO
- polynomial features degree > 1

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import time

## load raw

In [ ]:
from src.features.build_features import load_raw

df_all = load_raw()

[(x, df_all[x].shape) for x in df_all.keys()]

## load 5.1 preprocessed data

In [ ]:
# convert cell to execute (safety check)
df_targ = pd.read_pickle('data/processed/5.1B-df_targ.pkl')
df_feat_2 = pd.read_pickle('data/processed/5.1B-df_feat_2.pkl')
df_meta = pd.read_pickle('data/processed/5.1B-df_meta.pkl')

In [ ]:
df_feat_2.shape, df_targ.shape, df_meta.shape

In [ ]:
len(df_feat_2.columns), len(df_all['features_train'].columns)

In [ ]:
df_feat_2.columns, df_all['features_train'].columns

## selected features

In [ ]:
# features selected from
# https://github.com/drivendata/benchmarks/blob/master/dengue-benchmark-statsmodels.ipynb
#selected_features = ['reanalysis_specific_humidity_g_per_kg', 
#                 'reanalysis_dew_point_temp_k', 
#                 'station_avg_temp_c', 
#                 'station_min_temp_c']

# all features
# selected_features = df_all['features_train'].columns
# selected_features = df_feat_2.columns

# all that make sense
selected_features = [x for x in df_feat_2.columns
                     if (x.endswith('_trend') and not x.startswith('weekofyear'))
                    or x=='weekofyear_original']

# from RF feature importances (notebook 3.0)
#selected_features = [
#    'ndvi_sw',
#    'ndvi_se',
#    'reanalysis_dew_point_temp_k',
#    'reanalysis_specific_humidity_g_per_kg',
#    'station_max_temp_c',
#    'weekofyear',
#]
#selected_features = list(build_colnames(selected_features).values())

# from RF with diff (notebook 3.0.1)
#selected_features = [
#       'ndvi_sw',
#       'ndvi_se',
#       'reanalysis_avg_temp_k', # missing in 3.0
#       'reanalysis_dew_point_temp_k',
#       'reanalysis_specific_humidity_g_per_kg',
#       'station_avg_temp_c', # missing in 3.0
#       'station_max_temp_c',
#       'weekofyear',
#]
#selected_features = list(build_colnames(selected_features).values())

assert len(set(selected_features) - set(df_feat_2.columns))==0

selected_features

## train/test split

In [ ]:
df_train = df_feat_2.loc[~df_meta['submit']]

In [ ]:
# note avoiding class bias
x_train = (#df_all['features_train']
            df_train
          .groupby(level='city', as_index=False)
          .apply(lambda group: group.head(n=group.shape[0]*7//8))
          .reset_index(level=0, drop=True)
          [selected_features]
          )
x_test = (#df_all['features_train']
            df_train
          .groupby(level='city', as_index=False)
          .apply(lambda group: group.tail(n=group.shape[0]*1//8))
          .reset_index(level=0, drop=True)
          [selected_features]
         )
y_train = (#df_all['labels_train']
            df_targ.loc[~df_meta['submit']]
          .groupby('city', as_index=False)
          .apply(lambda group: group.head(n=group.shape[0]*7//8))
          .reset_index(level=0, drop=True)
          [['total_cases']]
         )
y_test = (#df_all['labels_train']        
            df_targ.loc[~df_meta['submit']]
          .groupby('city', as_index=False)
          .apply(lambda group: group.tail(n=group.shape[0]*1//8))
          .reset_index(level=0, drop=True)
          [['total_cases']]
)

In [ ]:
assert x_train.shape[0] == y_train.shape[0]
assert x_test.shape[0] == y_test.shape[0]

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
x_train.groupby('city').head(n=2)

In [ ]:
x_test.groupby('city').head(n=2)

In [ ]:
set(y_train.reset_index()['city'])

## log target

## normalize data to [-1,1]

In [ ]:
xtrain_scaled = x_train
xtest_scaled = x_test
ytrain_scaled = y_train
ytest_scaled = y_test

xtrain_scaled.shape, xtest_scaled.shape, ytrain_scaled.shape, ytest_scaled.shape

## LSTM params

In [ ]:
lahead = 60 # actually bad name .. should be look_back
batch_size = 64 # smaller batches lead to less loss of data when truncating non-multiples of batch_size

## create rolling windows for LSTM

In [ ]:
import numpy as np

In [ ]:
def stride_group(group, lahead):
    """
    https://pyfiddle.io/fiddle/a14865cf-38c9-48d6-b90a-f5bedc7a5b6e/
    """
    out = []
    for i in range(lahead):
        out.append(group.shift(i).values)
        
    out = np.stack(out, axis=2)[(lahead-1):, :, :] # drop first lahead
    out = np.swapaxes(out, 1, 2)
    out = np.flip(out, axis=1) # so that the index=0 is the oldest, and index=4 is latest
    return out

df_in = pd.DataFrame({'A': [1,2,3,4], 'B': [5,6,7,8]})
df_out = stride_group(df_in, 2)

print(df_in)
print(df_out)

In [ ]:
stride_group_2 = lambda x: stride_group(x, lahead=lahead)    
xtrain_roll = xtrain_scaled.groupby(level='city').apply(stride_group_2)
xtest_roll  = xtest_scaled.groupby(level='city').apply(stride_group_2)

In [ ]:
# for the target, drop first lahead points, without any rolling window
ytrain_roll = (ytrain_scaled
                .groupby(level='city', as_index=False)
                .apply(lambda group: group.iloc[lahead:])
                .reset_index(level=0, drop=True)
                )
ytest_roll = (ytest_scaled
              .groupby(level='city', as_index=False)
              .apply(lambda group: group.iloc[lahead:])
              .reset_index(level=0, drop=True)
             )

In [ ]:
[(city, [df.loc[city].shape for df in (xtrain_roll, xtest_roll, ytrain_roll, ytest_roll)]) for city in ['sj','iq']]

In [ ]:
ytrain_roll.groupby('city').size(), ytest_roll.groupby('city').size()

## WIP drop 1st x rows if they are not a multiple of batch_size


In [ ]:
for city in ['sj','iq']:
    xtrain_roll.loc[city] = xtrain_roll.loc[city][(xtrain_roll.loc[city].shape[0]%batch_size):]
    xtest_roll.loc[city] = xtest_roll.loc[city][(xtest_roll.loc[city].shape[0]%batch_size):]
    
def my_truncate(df):
    return (df.groupby(level='city', as_index=False)
              .apply(lambda group: group.tail(group.shape[0] - (group.shape[0]%batch_size)))
              .reset_index(level=0, drop=True)
            )

ytrain_roll = my_truncate(ytrain_roll)
ytest_roll = my_truncate(ytest_roll)
    

In [ ]:
ytrain_roll.groupby('city').size(), ytest_roll.groupby('city').size()

## verify data consistency between raw / scaled

## verify data consistency between scaled / roll

## fit LSTM

In [ ]:
# https://keras.io/layers/recurrent/#lstm
from keras.models import Sequential
from keras.layers import Dense, LSTM, Lambda, Dropout


In [ ]:
def create_model():
    model = Sequential()
    model.add(LSTM(100,
              input_shape=(lahead, len(selected_features)),
              batch_size=batch_size,
              activation='linear'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.add(Dropout(0.2))
    # scaling in a lambda layer so that the previous layer doesn't have to learn the scale much
    #model.add(Lambda(lambda x: x*10)) # TODO x*30 caused the re-fit on complete dataset to blow up
    model.compile(loss='mae', optimizer='adam')
    return model

## predict on test set

## re-fit on complete dataset

In [ ]:
# note avoiding class bias
x_retrain = df_train[selected_features]
y_retrain = df_targ.loc[~df_meta['submit']][['total_cases']]

xretrain_scaled = x_retrain
yretrain_scaled = y_retrain

xretrain_roll = xretrain_scaled.groupby(level='city').apply(stride_group_2)
yretrain_roll = (yretrain_scaled
                 .groupby(level='city', as_index=False)
                 .apply(lambda group: group.iloc[lahead:])
                 .reset_index(level=0, drop=True)
                )

# drop lahead per city
for city in ['sj','iq']:
    xretrain_roll.loc[city] = xretrain_roll.loc[city][(xretrain_roll.loc[city].shape[0]%batch_size):]
    
yretrain_roll = my_truncate(yretrain_roll)

y_retrain.groupby('city')['total_cases'].plot()
plt.show()

In [ ]:
yretrain_roll = np.log10(yretrain_roll+1)

for city in ['sj','iq']:
    yretrain_roll.loc[city]['total_cases'].plot()
plt.show()

In [ ]:
xretrain_roll.loc['sj'].shape

In [ ]:
plt.plot(xretrain_roll.loc['sj'][:,0,3], label='t0')
plt.plot(xretrain_roll.loc['sj'][:,10,3], label='t10')
plt.legend()
plt.show()

In [ ]:
mod1 = {}

In [ ]:
for city in ['sj', 'iq']:
    print(city)
    #if city=='sj': continue # FIXME fitting sj model
    mod1[city] = create_model()
    #if city=='iq': continue # FIXME skipping iq model
    mod1[city].summary()
    print(time.ctime(),'fit start')
    history = mod1[city].fit(
             xretrain_roll.loc[city],
             yretrain_roll.loc[city],
             batch_size=batch_size,
             epochs=500, # 1000,
             verbose=0,
             #validation_data=None,
             shuffle=False)
    print(time.ctime(),'fit end')
    
    plt.plot(history.history['loss'], label='loss')
    #plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend()
    plt.title(city)
    plt.show()

## set in submission

In [ ]:
df_all['submission'].loc['sj'].head(n=2)

In [ ]:
x_submit = df_feat_2.loc[ df_meta['submit'], selected_features]
xsubmit_scaled = x_submit
xsubmit_roll = xsubmit_scaled.groupby(level='city').apply(stride_group_2)

# drop non-batch_size multiple
for city in ['sj','iq']:
    xsubmit_roll.loc[city] = xsubmit_roll.loc[city][(xsubmit_roll.loc[city].shape[0]%batch_size):]

In [ ]:
predictions = (df_targ.loc[ df_meta['submit']].copy()*0).astype('int')[['total_cases']]

def my_predict(city):
    np_pred = mod1[city].predict(xsubmit_roll.loc[city], batch_size=batch_size)
    d1 = predictions.loc[city].shape[0]
    d2 = xsubmit_roll.loc[city].shape[0]
    return np.concatenate([np.zeros((d1-d2,1)), np_pred], axis=0)

predictions.loc['sj', 'total_cases'] = my_predict('sj')
predictions.loc['iq', 'total_cases'] = my_predict('iq')

# reverse log10 transform
predictions['total_cases'] = ((10**((predictions['total_cases']).clip(upper=3)))-1)


# FIXME cannot really apply scaler_ytest on the predictions
#predictions.loc[:] = scaler_ytest.inverse_transform(predictions).astype(int)
predictions['total_cases'] = predictions['total_cases'].astype(int)

In [ ]:
predictions.groupby('city').tail(n=2)

In [ ]:
submit = df_all['submission'].copy()
submit = submit.merge(predictions, how='left', left_index=True, right_index=True, suffixes=['_zero', ''])
del submit['total_cases_zero']
submit = submit.fillna(value=0)
submit['total_cases'] = submit['total_cases'].astype('int')
submit.groupby('city').tail(n=2)

## plot

In [ ]:
for city in ['sj','iq']:
    submit.loc[city, 'total_cases'].plot(figsize=(20,3), label=city)
    
plt.legend()
plt.show()

## generate submission file

In [ ]:
from src.features.build_features import make_submission

In [ ]:
make_submission(submit.reset_index())